In [1]:

import math
import os
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

pd.set_option('display.max_columns', None)
torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

#Environment Setup

In [2]:
# Turn to False to run locally
USE_GOOGLE_COLAB = False

if USE_GOOGLE_COLAB:
  # Mount MyDrive/QCEH_data/ to fetch training and testing data
  from google.colab import drive

  drive.mount('/content/drive')
  import sys
  folder_name = 'drive/MyDrive/QCEH_data/'
  sys.path.append(folder_name)
  %cd 'drive/MyDrive/QCEH_data/'
else:
  %cd 'QCEH_data/'

file_names = [f for f in os.listdir('./') if f.endswith('.parquet')]
file_names.sort()
print(file_names)

parquet_filename = 'TCV_DATAno64467.parquet'

/Users/ericsaikali/Library/Mobile Documents/com~apple~CloudDocs/Documents/GitHub/tokamak-unsupervised/QCEH_data
['TCV_DATAno61056.parquet', 'TCV_DATAno61057.parquet', 'TCV_DATAno64438.parquet', 'TCV_DATAno64467.parquet', 'TCV_DATAno64469.parquet', 'TCV_DATAno64495.parquet', 'TCV_DATAno64950.parquet', 'TCV_DATAno66166.parquet', 'TCV_DATAno66169.parquet', 'TCV_DATAno70302.parquet', 'TCV_DATAno70305.parquet', 'TCV_DATAno70306.parquet', 'TCV_DATAno70310.parquet', 'TCV_DATAno70311.parquet', 'TCV_DATAno70313.parquet', 'TCV_DATAno70654.parquet', 'TCV_DATAno70656.parquet', 'TCV_DATAno70657.parquet', 'TCV_DATAno71344.parquet', 'TCV_DATAno71345.parquet', 'TCV_DATAno71351.parquet', 'TCV_DATAno73532.parquet', 'TCV_DATAno73784.parquet', 'TCV_DATAno73785.parquet', 'TCV_DATAno73786.parquet', 'TCV_DATAno73838.parquet', 'TCV_DATAno73846.parquet', 'TCV_DATAno75461.parquet', 'TCV_DATAno75464.parquet', 'TCV_DATAno78058.parquet', 'TCV_DATAno78061.parquet', 'TCV_DATAno78064.parquet', 'TCV_DATAno78069.parque

/Users/ericsaikali/Library/Mobile Documents/com~apple~CloudDocs/Documents/GitHub/tokamak-unsupervised/venv/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


# Constants for trainings

In [3]:
# Setup device-agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [4]:
file_names = [f for f in os.listdir('./') if f.endswith('.parquet')]
file_names.sort()

test_shot_names = {
    "TCV_DATAno70310.parquet",
    "TCV_DATAno70311.parquet",
    "TCV_DATAno73838.parquet",
    "TCV_DATAno78603.parquet",
    "TCV_DATAno78604.parquet",
    "TCV_DATAno78606.parquet",
    "TCV_DATAno78608.parquet",
    "TCV_DATAno78611.parquet",
    "TCV_DATAno78637.parquet",
    "TCV_DATAno78639.parquet"
}

train_filenames = []
test_filenames = []
for file in file_names:
  if file in test_shot_names:
    test_filenames.append(file)
  else:
    train_filenames.append(file)

print(train_filenames)
print(test_filenames)
print(len(train_filenames), len(test_filenames), len(train_filenames) + len(test_filenames))

['TCV_DATAno61056.parquet', 'TCV_DATAno61057.parquet', 'TCV_DATAno64438.parquet', 'TCV_DATAno64467.parquet', 'TCV_DATAno64469.parquet', 'TCV_DATAno64495.parquet', 'TCV_DATAno64950.parquet', 'TCV_DATAno66166.parquet', 'TCV_DATAno66169.parquet', 'TCV_DATAno70302.parquet', 'TCV_DATAno70305.parquet', 'TCV_DATAno70306.parquet', 'TCV_DATAno70313.parquet', 'TCV_DATAno70654.parquet', 'TCV_DATAno70656.parquet', 'TCV_DATAno70657.parquet', 'TCV_DATAno71344.parquet', 'TCV_DATAno71345.parquet', 'TCV_DATAno71351.parquet', 'TCV_DATAno73532.parquet', 'TCV_DATAno73784.parquet', 'TCV_DATAno73785.parquet', 'TCV_DATAno73786.parquet', 'TCV_DATAno73846.parquet', 'TCV_DATAno75461.parquet', 'TCV_DATAno75464.parquet', 'TCV_DATAno78058.parquet', 'TCV_DATAno78061.parquet', 'TCV_DATAno78064.parquet', 'TCV_DATAno78069.parquet', 'TCV_DATAno78089.parquet', 'TCV_DATAno78090.parquet', 'TCV_DATAno78091.parquet', 'TCV_DATAno78104.parquet', 'TCV_DATAno78260.parquet', 'TCV_DATAno78261.parquet', 'TCV_DATAno78262.parquet', 

In [5]:
GENERAL_INPUTS = ["shotnumber", "time"]
MACHINE_INPUTS = ["isbaffled", "ip","b0","nel","ptot","pdiv","q95","betan","kappa","deltaavg","deltaupp",
                  "deltalow","gapin","gapout","zmag","rmag","rmin","lpar_ot","zeff"]
LABEL = ["LHD_label"]
INPUTS = GENERAL_INPUTS + MACHINE_INPUTS + LABEL

MAX_SHOT_LENGTH = 11001

input_size = len(MACHINE_INPUTS)
output_size = 4

batch_size = 10

len(MACHINE_INPUTS)

19

# Data preprocessing

In [6]:
def pad_dataframe(data, to_length, columns=INPUTS):
  # Padd shot' samples with dummy values at the beginning if the shot is too short
  if to_length != None and data.shape[0] < to_length:
    df = pd.DataFrame(0, index=np.arange(to_length - data.shape[0]), columns=columns)
    data = pd.concat([df, data], axis=0, ignore_index=True)
  return data

In [7]:
def read_data(file_names, expected_length=None):
  df_list = [
      pad_dataframe(
          pd.read_parquet(x).drop(columns=['alpha', 'H98y2calc'], errors='ignore'),
          expected_length
      )
      for x in file_names]
  df_list = pd.concat(df_list, ignore_index=True)
  df_list = df_list[MACHINE_INPUTS + LABEL]

  X = df_list.drop(["LHD_label"], axis=1)
  y = df_list["LHD_label"]

  return df_list, X, y

def analyse_data(file_names):
  _, X, y = read_data(file_names, MAX_SHOT_LENGTH)

  x_mean = np.mean(X, axis=0)
  x_std = np.std(X, axis=0)

  y_compared = np.arange(output_size)[:,np.newaxis] == np.array(y)
  labels_distribution = np.sum(y_compared, axis=1) / y.shape[0]

  return torch.tensor(x_mean, dtype=torch.float32), torch.tensor(x_std, dtype=torch.float32), torch.tensor(labels_distribution, dtype=torch.float32)

x_mean, x_std, labels_distribution = analyse_data(train_filenames)
print("labels distribution: ", labels_distribution)

torch.save(x_mean, "x_mean.pt")
torch.save(x_std, "x_std.pt")
torch.save(labels_distribution, "labels_distribution.pt")

labels distribution:  tensor([0.0360, 0.2290, 0.2247, 0.5103])


/var/folders/vq/vpp5gf_96v7f5w4frq2kpfn00000gn/T/ipykernel_10709/3394722215.py:25: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return torch.tensor(x_mean, dtype=torch.float32), torch.tensor(x_std, dtype=torch.float32), torch.tensor(labels_distribution, dtype=torch.float32)


# Loaders for datasets

In [8]:
class QCEH_Dataset(Dataset):
  def __init__(self, file_names, sequence_length, mean, std, transform=None):
    self.samples = []
    self.labels = []

    for file_name in file_names:
      _, X, y = read_data([file_name], sequence_length)
      X = torch.tensor(X.values, dtype=torch.float32)
      y = torch.tensor(y.values, dtype=torch.float32)
      X_normalized = (X - torch.unsqueeze(mean, 0)) / torch.unsqueeze(std, 0)

      if transform:
        X_normalized = transform(X_normalized)

      self.samples.append(X_normalized.to(device))
      self.labels.append(y.to(device))

  def __len__(self):
    return len(self.samples)

  def __getitem__(self, idx):
    return self.samples[idx], self.labels[idx]

train_data = QCEH_Dataset(
    train_filenames,
    MAX_SHOT_LENGTH,
    x_mean,
    x_std
)

test_data = QCEH_Dataset(
    test_filenames,
    MAX_SHOT_LENGTH,
    x_mean,
    x_std
)

train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [9]:
class QCEH_Dataset_Downsampled(Dataset):
  def __init__(self, file_names, mean, std, step_size, sub_sequence_length, sequence_length, transform=None, record=False):
    self.step_size = step_size
    self.sequence_length = sequence_length
    self.sub_sequence_length = sub_sequence_length

    self.samples = []
    self.labels = []

    for file_name in file_names:
      _, X, y = read_data([file_name], sequence_length)
      X = torch.tensor(X.values, dtype=torch.float32)
      y = torch.tensor(y.values, dtype=torch.float32)
      X_normalized = (X - torch.unsqueeze(mean, 0)) / torch.unsqueeze(std, 0)

      if transform:
        X_normalized = transform(X_normalized)

      self.samples.append(X_normalized)
      self.labels.append(y)

    if record:
      self.record = torch.zeros((output_size), dtype=torch.float32)

  def __len__(self):
    return len(self.samples)

  def __getitem__(self, idx):
    X = self.samples[idx]

    sub_sequence_id = random.randint(0, self.step_size - 1)
    sub_sequence_ids = torch.arange(0, self.sequence_length)
    sub_sequence_ids = sub_sequence_ids[
        sub_sequence_ids % self.step_size == sub_sequence_id
        ]
    sub_sequence_ids = sub_sequence_ids[:self.sub_sequence_length]

    samples = self.samples[idx][sub_sequence_ids,:]
    labels = self.labels[idx][sub_sequence_ids]

    if self.record != None:
      for i in range(output_size):
        self.record[i] += torch.sum(labels == i)

    samples = samples.to(device)
    labels = labels.to(device)

    return samples, labels

  def get_record(self):
    return self.record.clone().detach()

downsample_step = 11
downsample_sequence_size = math.floor(MAX_SHOT_LENGTH / downsample_step)

downsampled_training_data = QCEH_Dataset_Downsampled(
    train_filenames,
    x_mean,
    x_std,
    downsample_step,
    downsample_sequence_size,
    MAX_SHOT_LENGTH,
    record=True
)

downsampled_test_data = QCEH_Dataset_Downsampled(
    test_filenames,
    x_mean,
    x_std,
    downsample_step,
    downsample_sequence_size,
    MAX_SHOT_LENGTH,
    record=True
)

downsampled_train_dataloader = DataLoader(downsampled_training_data, batch_size=batch_size, shuffle=True)
downsampled_test_dataloader = DataLoader(downsampled_test_data, batch_size=batch_size, shuffle=True)

#General definitions for RNN models

In [10]:
"""
Function constructing a multiclass confusion matrix

:param y_predicted: pd.Series containing the predicted labels
:param y_expected: pd.Series containing the expected labels
:return: pd.Dataframe containing the multiclass confusion matrix
with columns as true labels and rows are predicted labels
"""
def confusion_matrix(y_predicted, y_expected, display_matrix=False):
  predicted_labels = y_predicted.unique()
  expected_labels = np.sort(y_expected.unique())

  mtrx = pd.DataFrame(0, columns=expected_labels, index=expected_labels)

  for expected_label in expected_labels:
    for predicted_label in predicted_labels:
      count = sum((y_expected == expected_label) & (y_predicted == predicted_label))
      mtrx.at[predicted_label, expected_label] = count

  if (display_matrix):
    display(mtrx)

  return mtrx

"""
Function retrieving the true values of a confusion matrix

:param matrix: confusion matrix
:return: list of all the diagonal values
"""
def diag(matrix):
  if len(matrix.index) <= len(matrix.columns):
    zipped = zip(matrix.index, matrix.columns[:len(matrix.index)])
  else:
    zipped = zip(matrix.index[:len(matrix.columns)], matrix.columns)

  diag = []
  for idx,col in zipped:
    diag.append(matrix.at[idx,col])
  return diag

"""
Function computing the precision associated with a confusion matrix
:param matrix: confusion matrix
:return: precision score
"""
def multi_precision(matrix, display_result=False):
  tp = diag(matrix)
  pred_p = matrix.sum(axis=1)
  if display_result:
    display(pred_p)
  return (tp / pred_p).fillna(0)

"""
Function computing the recall associated with a confusion matrix
:param matrix: confusion matrix
:return: recall score
"""
def multi_recall(matrix, display_result=False):
  tp = diag(matrix)
  acutal_p = matrix.sum(axis=0)
  if display_result:
    display(acutal_p)
  return (tp / acutal_p).fillna(0)

"""
Function computing the F1-score associated with a confusion matrix
:param matrix: confusion matrix
:return: F1-score
"""
def multi_f1_score(matrix, display_result=False):
  recall = multi_recall(matrix, display_result)
  precision = multi_precision(matrix, display_result)

  multiplied = recall.multiply(precision)
  summed = recall + precision
  return (2 * multiplied / summed).fillna(0)

In [11]:
def decode_output_to_states(output):
  # Convert predictions to probability of each class
  prob = F.softmax(output, dim=1).data
  # Taks the class with the highest probability score from the output
  output_states = torch.max(prob, dim=1)
  return output_states[1]

def run_model(model, output_size, dataloader, loss_fn, optimizer=None, display_matrix=False):
  loss_total = 0

  y_predicted = torch.empty((0,), dtype=torch.float32).to(device)
  y_labels = torch.empty((0,), dtype=torch.float32).to(device)

  for x, y in dataloader:
    if optimizer:
      optimizer.zero_grad()

    x = x.to(device)
    y = y.to(device)

    output, hidden = model(x)
    output_states = decode_output_to_states(output)

    loss = loss_fn(output, y.long().view(-1))

    if optimizer:
      loss.backward()
      optimizer.step()

    loss_total += loss

    # Records outputs for confusion matrix computation
    y_predicted = torch.cat([y_predicted, torch.reshape(output_states, (-1,))])
    y_labels = torch.cat([y_labels, torch.reshape(y, (-1,))])

  # Confusion matrix & f1 score
  matrix = confusion_matrix(pd.Series(y_predicted.cpu()), pd.Series(y_labels.cpu()), display_matrix=display_matrix)
  multi_f1 = multi_f1_score(matrix)

  return loss_total.item(), matrix, multi_f1


def train_model(model, output_size, dataloader, optimizer, loss_fn, n_epochs):
  model.train()

  loss_history = []
  f1_scores = []
  for out in range(output_size):
    f1_scores.append([])

  # Training Run
  for epoch in range(0, n_epochs):
    loss, _, multi_f1 = run_model(model, output_size, train_dataloader, loss_fn,
                                  optimizer=optimizer, display_matrix=(epoch == n_epochs - 1))

    loss_history.append(loss)
    for out in range(output_size):
      f1_scores[out].append(multi_f1[out].item())

    if epoch == 0:
      print('Training... [', end='')
    if epoch%10 == 0:
      print('#', end='')
    if epoch == n_epochs - 1:
      print('] done!')
      print('Epoch: {}/{}.............'.format(epoch + 1, n_epochs), end=' ')
      print("Loss: {:.4f}".format(loss))
      for out in range(output_size):
        print("\t state {}: F1 score = {:.4f}".format(out, f1_scores[out][-1]))

  # Plot results
  fig, ax = plt.subplots(1, 1)
  ax.plot(loss_history, 'm', label="Loss")
  ax.set_xlabel("Epoch")
  ax.set_ylabel("Loss")
  ax2 = ax.twinx()
  ax2.set_ylabel('F1 score')
  for out in range(output_size):
    ax2.plot(f1_scores[out], label='F1 score state {}'.format(out))
  ax.legend(loc="upper left")
  ax2.legend(loc="upper right")
  plt.show()

def test_model(model, output_size, lossFn, test_dataloader):
  model.eval()

  loss, _, multi_f1 = run_model(model, output_size, test_dataloader, lossFn, display_matrix=True)

  print("Evaluating model on testing data:")
  print("Loss: {:.4f}".format(loss))
  for out in range(output_size):
    print("\t state {}: F1 score = {:.4f}".format(out, multi_f1[out]))

In [12]:
cross_entropy = nn.CrossEntropyLoss(weight=labels_distribution.to(device))

#hyperparameters
n_epochs_rnn = 400
n_epochs_lstm = 400

# RNN Training

In [13]:
# create our RNN based network with an RNN followed by a linear layer
class RNN(nn.Module):
    def __init__(self, input_size, output_size, hidden_size, n_layers, drop_prob):
        super().__init__()

        self.hidden_dim = hidden_size
        self.n_layers = n_layers

        # Defining the layers
        # RNN Layer
        self.RNN = nn.RNN(input_size=input_size,
                          hidden_size=hidden_size,
                          num_layers=n_layers,
                          nonlinearity='tanh',
                          batch_first=True,
                          dropout=drop_prob)
        # Dropout layer
        self.dropout = nn.Dropout(drop_prob)
        # Fully connected layer
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
      batch_size = x.size(0)

      # Initializing hidden state for first input using method defined below
      hidden = self.init_hidden(batch_size)

      # Passing in the input and hidden state into the model and obtaining outputs
      out, hidden = self.RNN(x, hidden)

      # Passing trough the dropout layer
      out = self.dropout(out)

      # Reshaping the outputs such that it can be fit into the fully connected layer
      out = out.contiguous().view(-1, self.hidden_dim)
      out = self.fc(out)

      return out, hidden

    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim, dtype=torch.float32)
        hidden = hidden.to(device)
        return hidden

In [14]:
def train_RNN_model(input_size, output_size, num_layers, hidden_size, drop_prob, learning_rate, n_epochs, train_dataloader, loss_fn):
  print("---------------------------------------")
  print("Training RNN model with hyperparameters:")
  print("hidden_size = ", hidden_size)
  print("num_layers = ", num_layers)
  print("drop_prob = ", drop_prob)
  print("learning_rate = ", learning_rate)
  print("n_epochs = ", n_epochs)

  # Create our network instance, pick loss function and optimizer
  model = RNN(input_size, output_size, hidden_size, num_layers, drop_prob)
  model = model.to(device)

  # Define Loss, Optimizer
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

  # Train model
  train_model(model, output_size, train_dataloader, optimizer, loss_fn, n_epochs)

  return model

In [ ]:
for learning_rate_exp in range(-5, -1):
  learning_rate = 10 ** learning_rate_exp

  for num_layers_exp in range(0, 4):
    num_layers = 2 ** num_layers_exp

    for hidden_size_exp in range(4, 7):
      hidden_size = 2 ** hidden_size_exp

      for drop_prob in [0.2, 0.5, 0.7]:
        rnn_trained_model = train_RNN_model(input_size, output_size, num_layers, hidden_size,
                                      drop_prob, learning_rate, n_epochs_rnn, train_dataloader, cross_entropy)
        if num_layers == 1:
          break

        test_model(rnn_trained_model, output_size, cross_entropy, test_dataloader)

        model_name = "./rnn_10^{}_{}_{}_{:.1f}".format(learning_rate_exp, num_layers, hidden_size_exp, drop_prob)
        torch.save(rnn_trained_model.state_dict(), model_name)

        print(model_name, " is trained !")
        print("\n\n")

# LSTM Training

In [16]:
class LSTM(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers, drop_prob=0.5):
        super().__init__()

        # Defining some parameters
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

        #Defining the layers
        # RNN Layer
        self.lstm = nn.LSTM(input_size, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x):
        batch_size = x.size(0)

        # Initializing hidden state for first input using method defined below
        hidden = self.init_hidden(batch_size)

        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.lstm(x, hidden)
        out = out.contiguous().view(-1, self.hidden_dim)

        # Passing trough the dropout layer
        out = self.dropout(out)

        # Reshaping the outputs such that it can be fit into the fully connected layer
        out = self.fc(out)

        return out, hidden

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        return hidden

def train_LSTM_model(input_size, output_size, hidden_size, num_layers, learning_rate, drop_prob, n_epochs, train_dataloader, loss_fn):
  print("---------------------------------------")
  print("Training LSTM model with hyperparameters:")
  print("hidden_size = ", hidden_size)
  print("num_layers = ", num_layers)
  print("learning_rate = ", learning_rate)
  print("drop_prob = ", drop_prob)
  print("n_epochs = ", n_epochs)

  # Create our network instance, pick loss function and optimizer
  model = LSTM(input_size, output_size, hidden_size, num_layers)
  model = model.to(device)

  # Define Loss, Optimizer
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

  # Train model
  train_model(model, output_size, train_dataloader, optimizer, loss_fn, n_epochs)

  return model

In [ ]:
for learning_rate_exp in range(-5, -1):
  learning_rate = 10 ** learning_rate_exp

  for num_layers_exp in range(1, 3):
    num_layers = 2 ** num_layers_exp

    for hidden_size_exp in range(5, 8):
      hidden_size = 2 ** hidden_size_exp

      for drop_prob in [0.2, 0.5, 0.7]:
        ltsm_trained_model = train_LSTM_model(input_size, output_size, hidden_size,
                                      num_layers, learning_rate, drop_prob,
                                      n_epochs_lstm, downsampled_train_dataloader, cross_entropy)

        test_model(ltsm_trained_model, output_size, cross_entropy, downsampled_test_dataloader)

        model_name = "./ltsm_10^{}_{}_{}_{:.1f}".format(learning_rate_exp, num_layers, hidden_size_exp, drop_prob)
        torch.save(ltsm_trained_model.state_dict(), model_name)

        print(model_name, " is trained !")
        print("\n\n")